## 🚀 Day2-01：Agent Tools

5 天 AI Agents 强化课程的第二天，我们将学习如何使用工具来扩展智能体的功能。

回顾：在第一天的课程中，学会了如何利用内置工具创建智能体，并掌握了多 Agent 系统的协调方法。

今天，我们将通过构建自定义逻辑、委托专业智能体以及处理现实世界中的复杂问题，来解锁智能体工具的全部潜力。

## 为什么 Agents 需要 Tools？

问题所在：没有工具，代理的知识就会停留在过去，它无法获取今天最新的新闻或公司库存信息。即使是最先进的基础模型，如果没有外部函数的访问权限，本质上仍然只是一个模式预测引擎，它无法访问任何关于世界的最新数据。

解决方案：

（1）Tools 可以克服基础模型的固有局限性。

（2）Tools 赋予 Agents “感知”和“行动”的能力。使你孤立的 LLM 转变为能够真正帮助你完成任务的强大 Agents。

Day2 任务，你将掌握：
- ✅ 将Python函数转化为代理工具
- ✅ 构建一个代理，并将其作为另一个代理的工具使用
- ✅ 构建你的第一个多工具代理
- ✅ 探索ADK中的不同工具类型

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

入门步骤：

1.vscode 安装 jupyter 插件

# Section 1：设置环境

##### 1.1 配置你的 Gemini API密钥

In [73]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key setup complete.")
            break
else:
    print(f"⚠️ Please create .env file at: {env_file}")


✅ Gemini API key setup complete.


##### 1.2 导入 ADK 组件

基于 Google 的 Agent Development Kit (ADK) 框架来构建 Agent。ADK 提供了一组工具和库，用于构建和运行 Agent。

In [74]:
from google.genai import types # 基础类型定义

from google.adk.agents import LlmAgent # agent 核心逻辑框架
from google.adk.models.google_llm import Gemini #具体模型实现
from google.adk.runners import InMemoryRunner # 负责在一个循环中处理交互：接收用户输入 -> 传给 Agent -> 等待 Agent 思考/调用工具 -> 返回结果。
from google.adk.sessions import InMemorySessionService # 会话管理
from google.adk.tools import google_search, AgentTool, ToolContext # 工具箱
from google.adk.code_executors import BuiltInCodeExecutor # 执行器

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


##### 1.3 辅助函数

可选的，用于打印生成的Python代码及代码执行工具运行结果的辅助函数：

In [75]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


##### 1.4：配置重试选项

当你在使用LLM时，可能会遇到瞬时错误，如速率限制或临时服务不可用。重试选项通过指数退避自动重试请求来处理这些故障。

In [76]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 🤖 Section 2: 什么是自定义工具？

自定义工具是你可以根据自己的代码和业务逻辑构建的工具，让你完全掌控它的功能。

##### 2.1：构建自定义 Tools

Example: 电商售后客服
- get_order_status (订单状态查询) - 连接数据库查询订单状态。
- process_refund (退款处理) - 连接支付网关执行退款。
- cancel_shipping (取消物流) - 通知物流公司拦截包裹。



![](:https://fisherai-1312281807.cos.ap-guangzhou.myqcloud.com/202511182341285.png)

In [77]:
# 模拟数据库 - 订单信息
ORDERS_DB = {
    "8823": {
        "order_id": "8823",
        "status": "未发货",
        "items": ["蓝牙耳机"],
        "total": 299.00,
        "payment_status": "已支付"
    },
    "8824": {
        "order_id": "8824", 
        "status": "已发货",
        "items": ["无线鼠标"],
        "total": 159.00,
        "payment_status": "已支付",
        "tracking_number": "YTO123456789"
    }
}

print("✅ 模拟订单数据库初始化完成")

✅ 模拟订单数据库初始化完成


In [78]:
# 工具 1: 查询订单状态
def get_order_status(order_id: str) -> dict:
    """
    查询订单状态
    
    Args:
        order_id: 订单号
        
    Returns:
        包含订单详细信息的字典
    """
    if order_id in ORDERS_DB:
        return {
            "success": True,
            "data": ORDERS_DB[order_id]
        }
    else:
        return {
            "success": False,
            "error": f"订单 #{order_id} 不存在"
        }

print("✅ 工具1: get_order_status 定义完成")

✅ 工具1: get_order_status 定义完成


In [79]:
# 工具 2: 处理退款
def process_refund(order_id: str, reason: str = "客户要求退款") -> dict:
    """
    处理订单退款
    
    Args:
        order_id: 订单号
        reason: 退款原因
        
    Returns:
        退款处理结果
    """
    if order_id not in ORDERS_DB:
        return {
            "success": False,
            "error": f"订单 #{order_id} 不存在"
        }
    
    order = ORDERS_DB[order_id]
    
    # 检查订单状态是否允许退款
    if order["status"] == "已发货":
        return {
            "success": False,
            "error": "订单已发货，请先取消物流后再申请退款"
        }
    
    # 模拟退款处理
    return {
        "success": True,
        "message": f"订单 #{order_id} 退款成功",
        "refund_amount": order["total"],
        "refund_time": "预计24小时到账",
        "reason": reason
    }

print("✅ 工具2: process_refund 定义完成")

✅ 工具2: process_refund 定义完成


In [80]:
# 工具 3: 取消物流
def cancel_shipping(order_id: str) -> dict:
    """
    取消订单物流配送
    
    Args:
        order_id: 订单号
        
    Returns:
        物流取消结果
    """
    if order_id not in ORDERS_DB:
        return {
            "success": False,
            "error": f"订单 #{order_id} 不存在"
        }
    
    order = ORDERS_DB[order_id]
    
    if order["status"] != "已发货":
        return {
            "success": False,
            "error": f"订单状态为'{order['status']}'，无需取消物流"
        }
    
    # 模拟取消物流
    tracking_number = order.get("tracking_number", "未知")
    return {
        "success": True,
        "message": f"已通知物流公司拦截包裹",
        "tracking_number": tracking_number,
        "note": "物流拦截需要1-2个工作日处理"
    }

print("✅ 工具3: cancel_shipping 定义完成")

✅ 工具3: cancel_shipping 定义完成


**测试自定义工具**

在将工具交给Agent使用之前，先单独测试每个工具函数，确保它们按预期工作。

In [81]:
# 测试 1: 查询订单状态
print("=== 测试1: 查询订单状态 ===")
print("查询订单 #8823:")
result = get_order_status("8823")
print(result)
print()

print("查询不存在的订单 #9999:")
result = get_order_status("9999")
print(result)

=== 测试1: 查询订单状态 ===
查询订单 #8823:
{'success': True, 'data': {'order_id': '8823', 'status': '未发货', 'items': ['蓝牙耳机'], 'total': 299.0, 'payment_status': '已支付'}}

查询不存在的订单 #9999:
{'success': False, 'error': '订单 #9999 不存在'}


In [82]:
# 测试 2: 处理退款
print("=== 测试2: 处理退款 ===")
print("订单 #8823 申请退款（未发货，应该成功）:")
result = process_refund("8823", "不想要了")
print(result)
print()

print("订单 #8824 申请退款（已发货，应该失败）:")
result = process_refund("8824")
print(result)

=== 测试2: 处理退款 ===
订单 #8823 申请退款（未发货，应该成功）:
{'success': True, 'message': '订单 #8823 退款成功', 'refund_amount': 299.0, 'refund_time': '预计24小时到账', 'reason': '不想要了'}

订单 #8824 申请退款（已发货，应该失败）:
{'success': False, 'error': '订单已发货，请先取消物流后再申请退款'}


In [83]:
# 测试 3: 取消物流
print("=== 测试3: 取消物流 ===")
print("取消订单 #8824 的物流（已发货，应该成功）:")
result = cancel_shipping("8824")
print(result)
print()

print("取消订单 #8823 的物流（未发货，应该失败）:")
result = cancel_shipping("8823")
print(result)

=== 测试3: 取消物流 ===
取消订单 #8824 的物流（已发货，应该成功）:
{'success': True, 'message': '已通知物流公司拦截包裹', 'tracking_number': 'YTO123456789', 'note': '物流拦截需要1-2个工作日处理'}

取消订单 #8823 的物流（未发货，应该失败）:
{'success': False, 'error': "订单状态为'未发货'，无需取消物流"}


##### 2.2：将Python函数转换为ADK工具

要让Agent能够使用这些工具，我们需要将它们转换为ADK可以识别的工具格式。ADK使用类型注解和文档字符串来理解工具的功能。

In [84]:
# 创建一个带有多个自定义工具的客服Agent
customer_service_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    tools=[
        get_order_status,      # 工具1：查询订单状态
        process_refund,        # 工具2：处理退款
        cancel_shipping,       # 工具3：取消物流
    ],
    name="customer_service_agent",
    description="电商售后客服智能助手，可以查询订单、处理退款和取消物流",
)

print("✅ 客服Agent创建成功，已配置3个自定义工具")

✅ 客服Agent创建成功，已配置3个自定义工具


##### 2.3：测试Agent与工具的交互

现在让我们创建一个运行环境，并测试Agent如何智能地调用这些工具来解决用户问题。

In [85]:
# 创建运行器
runner = InMemoryRunner(agent=customer_service_agent)

print("✅ Agent运行环境准备就绪")

✅ Agent运行环境准备就绪


In [86]:
# 测试场景1：用户咨询订单状态并申请退款
print("=" * 60)
print("场景1：订单 #8823 没发货，我要退款")
print("=" * 60)

response = await runner.run_debug("订单 #8823 没发货，我要退款")

for msg in response:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text:
                print(f"\n🤖 客服回复: {part.text}")

场景1：订单 #8823 没发货，我要退款

 ### Created new session: debug_session_id

User > 订单 #8823 没发货，我要退款


customer_service_agent > 好的，您的订单 #8823 退款已成功，退款金额299元，预计24小时到账。退款原因是：未发货。

🤖 客服回复: 好的，您的订单 #8823 退款已成功，退款金额299元，预计24小时到账。退款原因是：未发货。


In [87]:
# 测试场景2：已发货订单，需要先取消物流再退款
print("\n" + "=" * 60)
print("场景2：订单 #8824 已发货，我想退款")
print("=" * 60)

response = await runner.run_debug("订单 #8824 已发货，我想退款")

for msg in response:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text:
                print(f"\n🤖 客服回复: {part.text}")


场景2：订单 #8824 已发货，我想退款

 ### Continue session: debug_session_id

User > 订单 #8824 已发货，我想退款


customer_service_agent > 您的订单 #8824 已发货，无法直接退款。您可以选择先取消物流，然后再申请退款。您是否需要取消物流？

🤖 客服回复: 您的订单 #8824 已发货，无法直接退款。您可以选择先取消物流，然后再申请退款。您是否需要取消物流？


##### 2.4：关键要点总结

通过上面的示例，我们学到了：

1. **自定义工具的本质**：
   - 就是普通的Python函数
   - 需要有清晰的类型注解（参数和返回值）
   - 需要有详细的文档字符串（docstring）
   - ADK会自动将这些信息转换为LLM可理解的工具描述

2. **Agent如何使用工具**：
   - Agent会根据用户的需求，智能地选择调用哪个工具
   - 可以按需组合调用多个工具（如场景2中先取消物流再退款）
   - 根据工具返回的结果生成自然语言回复

3. **工具设计的最佳实践**：
   - ✅ 单一职责：每个工具做一件事
   - ✅ 清晰命名：函数名要能表达工具的功能
   - ✅ 详细文档：docstring要说明工具的用途、参数含义、返回值格式
   - ✅ 错误处理：返回结构化的成功/失败信息
   - ✅ 业务逻辑：工具内部可以包含复杂的业务规则（如订单状态检查）

**下一步**：了解更多高级工具类型，如代理作为工具(AgentTool)、代码执行工具等。

# 🔧 Section 3: 高级工具类型

接下来，我们将探索ADK中的其他高级工具类型：
- **AgentTool**: 将一个Agent封装成另一个Agent的工具
- **ToolContext**: 访问会话上下文和状态
- **BuiltInCodeExecutor**: 让Agent能够执行Python代码
- **InMemorySessionService**: 管理多轮对话的会话状态

##### 3.1：AgentTool - 将Agent作为工具使用

AgentTool允许你将一个专业的Agent封装成工具，供其他Agent调用。这是构建多Agent系统的关键技术。

**使用场景**：当某个任务需要专业知识或复杂推理时，可以创建一个专门的Agent来处理。

In [88]:
# 创建一个专业的价格分析Agent
price_analyst_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    name="price_analyst",
    description="专业价格分析师，可以分析商品价格是否合理并提供建议",
    instruction="""
    你是一个专业的电商价格分析师。
    当收到商品信息时，分析价格是否合理，并给出专业建议。
    考虑因素：市场平均价、品质、品牌等。
    """,
)

# 将价格分析Agent封装成工具
price_analysis_tool = AgentTool(agent=price_analyst_agent)

# 创建一个增强版的客服Agent，集成价格分析功能
enhanced_customer_service_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    tools=[
        get_order_status,
        process_refund,
        cancel_shipping,
        price_analysis_tool,  # 新增：价格分析工具
    ],
    name="enhanced_customer_service",
    description="增强版电商客服，支持订单管理和价格咨询",
    instruction="""
    你是一个专业的电商客服。
    可以处理订单查询、退款、物流等问题。
    当客户咨询价格是否合理时，使用价格分析工具获得专业意见。
    """,
)

print("✅ 增强版客服Agent创建成功")
print("   - 包含3个自定义工具函数")
print("   - 包含1个AgentTool（价格分析专家）")

✅ 增强版客服Agent创建成功
   - 包含3个自定义工具函数
   - 包含1个AgentTool（价格分析专家）


In [89]:
# 测试AgentTool：咨询价格是否合理
runner2 = InMemoryRunner(agent=enhanced_customer_service_agent)

print("=" * 60)
print("测试AgentTool：咨询蓝牙耳机299元是否合理")
print("=" * 60)

response = await runner2.run_debug("我想买蓝牙耳机，299元的价格合理吗？")

for msg in response:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text:
                print(f"\n🤖 客服回复: {part.text}")

测试AgentTool：咨询蓝牙耳机299元是否合理

 ### Created new session: debug_session_id

User > 我想买蓝牙耳机，299元的价格合理吗？


enhanced_customer_service > 299元的价格对于蓝牙耳机来说，处于中等偏下的价位区间，算是一个比较常见的价格。

**在这个价位，您通常可以买到：**

*   **品质：** 普遍音质尚可，连接稳定，续航中等（单次使用4-6小时，配合充电盒总续航20-30小时），可能具备基础通话降噪（ENC）功能的真无线（TWS）蓝牙耳机。
*   **品牌：** 可能会是小米、红米、QCY、漫步者、Haylou、SoundPEATS、FIIL（入门级）等国产品牌，或者一些性价比较高的非一线国际品牌。

**要判断299元是否划算，还需要考虑以下关键因素：**

1.  **品牌和型号：**
    *   **非常划算：** 如果是小米、红米、漫步者、QCY等知名度较高、口碑不错的品牌，并且带有主动降噪（ANC）功能、佩戴检测、无线充电、低延迟游戏模式、防水等级（如IPX4/5）等一个或多个特色功能，那么299元是非常划算的。例如，一些入门级的主动降噪耳机，或者续航特别长的非降噪耳机，这个价格很不错。
    *   **合理范围：** 如果是上述品牌的基础款TWS耳机，不带主动降噪，但音质和稳定性表现良好，299元也是一个合理的价格。
    *   **可能不值：** 如果是完全不认识的“杂牌”或没有明确品牌信息的产品，即使功能宣称再好，299元也存在风险，可能音质、连接稳定性、续航和品控都无法保证。

2.  **核心功能：**
    *   **是否有主动降噪（ANC）？** 如果有，299元通常是个不错的价格。
    *   **通话降噪效果如何？** 对于经常通话的用户很重要。
    *   **延迟表现？** 玩游戏或看视频时，低延迟非常重要。
    *   **续航能力？** 单次和总续航时长。

3.  **音质表现：** 虽然299元很难达到发烧级别，但好的调音也能提供令人愉悦的听感。可以参考同价位产品的用户评价。

**建议：**

为了给您更精准的评估，请提供该蓝牙耳机的**具体品牌和型号**。这样我可以结合市场平均价、同类产品的配置和用户评价，给出更专业的购买建议。

**总结：** 299元本身并非一个离谱的价格，但其具体价值和性价比高低，完全取决于产品的品牌、型号和功能特性。

🤖 客服回复: 299元的价格对于蓝

##### 3.2：InMemorySessionService - 会话管理

InMemorySessionService用于管理多轮对话的会话状态，让Agent能够记住之前的对话内容。

In [90]:
# 创建会话服务
session_service = InMemorySessionService()

# 创建一个简单的客服Agent
session_demo_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    tools=[get_order_status, process_refund],
    name="session_demo_agent",
    description="支持多轮对话的客服Agent",
    instruction="""
    你是一个电商客服助手。
    当用户要求查询订单时，使用get_order_status工具查询订单信息。
    当用户要求退款时，先确认订单号，然后使用process_refund工具处理退款。
    在多轮对话中，记住之前提到的订单号。
    """,
)

# 创建使用会话服务的Runner
runner_with_session = InMemoryRunner(
    agent=session_demo_agent,
    app_name="customer_service_app",  # 应用名称
)

print("✅ 带会话管理的Agent创建成功")

✅ 带会话管理的Agent创建成功


In [91]:
# 测试会话管理：同一session中的多轮对话
print("=" * 60)
print("测试会话管理：多轮对话中Agent会记住上下文")
print("=" * 60)

# 使用新的session_id，避免与之前的session冲突
import time
session_id = f"customer_{int(time.time())}"

# 第一轮对话 - 查询订单
print("\n【第1轮对话 - 查询订单】")
response1 = await runner_with_session.run_debug(
    "帮我查一下订单8823的状态",
    session_id=session_id
)

# 只显示最后一条Agent的文本回复
final_text = None
for msg in response1:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text and len(part.text) > 10:
                final_text = part.text
                
if final_text:
    print(f"🤖 {final_text}\n")

# 第二轮对话 - 基于上下文申请退款
print("【第2轮对话 - 基于上下文申请退款】")
response2 = await runner_with_session.run_debug(
    "这个订单我不想要了，帮我退款吧",  # 没有指定订单号，Agent应该知道是8823
    session_id=session_id
)

# 只显示最后一条Agent的文本回复
final_text = None
for msg in response2:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text and len(part.text) > 10:
                final_text = part.text
                
if final_text:
    print(f"🤖 {final_text}")

测试会话管理：多轮对话中Agent会记住上下文

【第1轮对话 - 查询订单】

 ### Created new session: customer_1763483417

User > 帮我查一下订单8823的状态


session_demo_agent > 您的订单8823状态为未发货，已支付，商品是蓝牙耳机，总价299元。
🤖 您的订单8823状态为未发货，已支付，商品是蓝牙耳机，总价299元。

【第2轮对话 - 基于上下文申请退款】

 ### Continue session: customer_1763483417

User > 这个订单我不想要了，帮我退款吧


session_demo_agent > 您的订单8823退款已成功，退款金额299元，预计24小时到账。
🤖 您的订单8823退款已成功，退款金额299元，预计24小时到账。


**会话管理要点**：

✅ **session_id的作用**：
- 相同的session_id表示同一个会话
- Agent会记住该会话中的所有历史对话
- 在第2轮对话中，Agent能够理解"这个订单"指的是第1轮中提到的订单8823

✅ **改进点**：
- 添加了明确的instruction指导Agent行为
- 使用动态生成的session_id避免冲突
- 优化了输出逻辑，只显示最终的Agent回复

##### 3.3：综合应用 - 结合多种工具

现在让我们创建一个综合应用，结合自定义工具、内置工具（google_search）和AgentTool。

In [92]:
# 创建一个综合性的超级客服Agent
# 集成多种类型的工具

super_agent = LlmAgent(
    model=Gemini(model_id="gemini-2.0-flash-exp"),
    tools=[
        # 自定义工具函数
        get_order_status,
        process_refund,
        cancel_shipping,
        # AgentTool（专业子Agent）
        price_analysis_tool,
    ],
    name="super_customer_service",
    description="高级电商客服，集成订单管理和专业价格咨询能力",
    instruction="""
    你是一个高级的电商客服助手。你可以：
    1. 处理订单相关问题（查询、退款、取消物流）
    2. 调用专业的价格分析师为客户提供价格建议
    
    根据用户的需求，智能选择合适的工具来解决问题。
    """,
)

print("✅ 高级客服Agent创建成功")
print("   集成工具:")
print("   - 3个自定义函数工具（订单管理）")
print("   - 1个AgentTool（价格分析专家）")

✅ 高级客服Agent创建成功
   集成工具:
   - 3个自定义函数工具（订单管理）
   - 1个AgentTool（价格分析专家）


In [93]:
# 测试综合Agent：结合自定义工具和AgentTool
runner_super = InMemoryRunner(agent=super_agent)

print("=" * 60)
print("测试综合场景：订单查询 + 价格咨询")
print("=" * 60)

response = await runner_super.run_debug(
    "我的订单8823还没发货，里面是蓝牙耳机299元，这个价格合适吗？"
)

for msg in response:
    if msg.content and msg.content.parts:
        for part in msg.content.parts:
            if part.text and len(part.text) > 10:
                print(f"\n🤖 高级客服: {part.text}")
                break

测试综合场景：订单查询 + 价格咨询

 ### Created new session: debug_session_id

User > 我的订单8823还没发货，里面是蓝牙耳机299元，这个价格合适吗？


super_customer_service > 您好！您的订单8823目前状态是**未发货**，商品是蓝牙耳机，总价299元。

关于蓝牙耳机299元的价格，专业价格分析师的答复是：

299元对于蓝牙耳机来说是一个常见的价格区间，通常处于中低端市场或入门级中端市场。您可能会买到入门级大品牌或知名品牌的次级产品，或者优秀品质的非一线品牌，甚至少数品牌可能会在此价位推出带主动降噪功能的产品。

**为了提供更精准的建议，分析师需要您提供更多信息，例如：**
*   **品牌和具体型号**
*   **耳机类型**（真无线、颈挂式、头戴式）
*   **主要功能和特点**（音质、降噪、续航、防水等级、连接技术等）

**在您提供更多信息之前，大致判断标准是：**
*   如果无品牌或小品牌且无突出功能，299元可能偏高。
*   如果是知名品牌的基础款，具备稳定连接、清晰通话、6小时以上单次续航，299元是合理的价格。
*   如果是性价比品牌，且带有不错的音质、稳定的连接、8小时以上单次续航，甚至有基础主动降噪或IPX5以上防水，那么299元是一个非常值得考虑的价格。

如果您能提供更多详细信息，分析师会为您进行更详细的分析！

🤖 高级客服: 您好！您的订单8823目前状态是**未发货**，商品是蓝牙耳机，总价299元。

关于蓝牙耳机299元的价格，专业价格分析师的答复是：

299元对于蓝牙耳机来说是一个常见的价格区间，通常处于中低端市场或入门级中端市场。您可能会买到入门级大品牌或知名品牌的次级产品，或者优秀品质的非一线品牌，甚至少数品牌可能会在此价位推出带主动降噪功能的产品。

**为了提供更精准的建议，分析师需要您提供更多信息，例如：**
*   **品牌和具体型号**
*   **耳机类型**（真无线、颈挂式、头戴式）
*   **主要功能和特点**（音质、降噪、续航、防水等级、连接技术等）

**在您提供更多信息之前，大致判断标准是：**
*   如果无品牌或小品牌且无突出功能，299元可能偏高。
*   如果是知名品牌的基础款，具备稳定连接、清晰通话、6小时以上单次续航，299元是合理的价格。
*   如果是性价比品牌，且带有不错的音质、稳定的连接、8小时以上单次续航，甚至有基础主动降噪或IPX5以上防水，那么299元是一个非常值得考虑的价格。

如

# 📚 Section 4: Complete Guide to ADK Tool Types

现在你已经看到了工具的实际应用，让我们全面了解ADK工具包的完整分类体系。

## 4.1 ADK工具分类体系

ADK工具主要分为两大类：**Custom Tools（自定义工具）** 和 **Built-in Tools（内置工具）**

### 🔧 Custom Tools（自定义工具）

自定义工具是你根据业务需求自己编写的工具，提供最大的灵活性。

#### 1. Function Tools
- **描述**: 最常用的自定义工具类型，将Python函数转换为Agent工具
- **使用场景**: 
  - 数据库查询（订单查询、用户信息）
  - 业务逻辑处理（退款、发货、计算）
  - 外部API调用（支付网关、物流接口）
- **示例**: `get_order_status`, `process_refund`, `cancel_shipping`
- **特点**: 
  - ✅ 需要类型注解和docstring
  - ✅ 自动参数解析
  - ✅ 支持同步和异步函数

#### 2. Long Running Function Tools
- **描述**: 用于执行耗时较长的任务
- **使用场景**: 
  - 大数据处理
  - 文件上传/下载
  - 批量操作
- **特点**: 
  - ✅ 支持后台执行
  - ✅ 可追踪进度

#### 3. Agent Tools
- **描述**: 将一个专业Agent封装成工具供其他Agent使用
- **使用场景**: 
  - 多Agent协作系统
  - 专业领域分工（价格分析、法律咨询、技术支持）
- **示例**: `price_analyst_agent` 封装为 `price_analysis_tool`
- **特点**: 
  - ✅ 实现Agent分层架构
  - ✅ 复用专业Agent能力
  - ✅ 降低单个Agent的复杂度

#### 4. MCP Tools (Model Context Protocol)
- **描述**: 标准化的工具协议，便于工具共享和集成
- **使用场景**: 
  - 跨平台工具共享
  - 第三方工具集成
- **特点**: 
  - ✅ 标准化接口
  - ✅ 易于分发和维护

#### 5. OpenAPI Tools
- **描述**: 基于OpenAPI规范的工具，自动从API文档生成
- **使用场景**: 
  - 集成RESTful API
  - 第三方服务对接
- **特点**: 
  - ✅ 自动生成工具定义
  - ✅ 减少手动编码

---

### 🏗️ Built-in Tools（内置工具）

ADK提供了一系列开箱即用的内置工具，覆盖常见需求。

#### Gemini Tools（Gemini专属工具）

##### 1. google_search
- **描述**: Google搜索工具，获取实时互联网信息
- **使用场景**: 
  - 实时新闻查询
  - 产品信息搜索
  - 市场动态了解
- **特点**: 
  - ✅ 实时数据
  - ⚠️ 不能与自定义函数工具混用（AFC限制）

##### 2. BuiltInCodeExecutor
- **描述**: Python代码执行工具
- **使用场景**: 
  - 数据计算和分析
  - 动态代码生成和执行
  - 数学问题求解
- **特点**: 
  - ✅ 安全的沙箱环境
  - ⚠️ 不能直接作为tools参数使用
  - 📝 需要特殊配置方式

#### Google Cloud Tools（Google云服务工具）

##### 3. BigQueryToolset
- **描述**: BigQuery数据仓库查询工具
- **使用场景**: 
  - 大数据分析
  - 数据仓库查询
  - 商业智能报表

##### 4. SpannerToolset
- **描述**: Cloud Spanner数据库操作工具
- **使用场景**: 
  - 分布式数据库操作
  - 全球一致性事务

#### Third-party Tools（第三方工具）

##### 5. Github Tools
- **描述**: GitHub仓库操作工具
- **使用场景**: 
  - 代码仓库管理
  - Issue和PR处理
  - 代码审查

##### 6. HuggingFace Tools
- **描述**: HuggingFace模型和数据集工具
- **使用场景**: 
  - AI模型调用
  - 数据集访问
  - ML工作流集成

---

## 4.2 工具选择指南

```
选择Custom Tools的情况:
├── 需要业务特定逻辑 → Function Tools
├── 需要专业领域知识 → Agent Tools
├── 需要长时间运行 → Long Running Function Tools
├── 需要标准化接口 → MCP Tools
└── 已有OpenAPI文档 → OpenAPI Tools

选择Built-in Tools的情况:
├── 需要互联网搜索 → google_search
├── 需要代码执行 → BuiltInCodeExecutor
├── 需要Google云服务 → BigQueryToolset/SpannerToolset
└── 需要第三方集成 → Github/HuggingFace Tools
```

## 4.3 今日学习回顾

### ✅ 核心概念
1. **自定义工具（Custom Tools）**
   - 普通Python函数即可成为工具
   - 需要类型注解和详细docstring
   - ADK自动将函数转换为LLM可理解的工具描述

2. **AgentTool**
   - 将专业Agent封装成工具
   - 实现Agent之间的协作
   - 构建多Agent系统的关键技术

3. **InMemorySessionService**
   - 管理多轮对话的会话状态
   - 通过session_id区分不同用户
   - Agent能够记住上下文信息

### ✅ 完整工具对照表

| 工具类型 | 工具名称 | 分类 | 本课程使用 | 主要用途 |
|---------|---------|------|-----------|---------|
| **Custom Tools** |
| Function Tools | 自定义Python函数 | Custom | ✅ | 业务逻辑、数据操作 |
| Agent Tools | `AgentTool` | Custom | ✅ | Agent封装、多Agent协作 |
| Long Running Tools | 长时运行函数 | Custom | - | 耗时任务、后台处理 |
| MCP Tools | Model Context Protocol | Custom | - | 标准化工具接口 |
| OpenAPI Tools | OpenAPI规范工具 | Custom | - | API集成 |
| **Built-in Tools - Gemini** |
| Google Search | `google_search` | Built-in | ✅ 已导入 | 实时互联网搜索 |
| Code Executor | `BuiltInCodeExecutor` | Built-in | ✅ 已导入 | Python代码执行 |
| **Built-in Tools - Google Cloud** |
| BigQuery | `BigQueryToolset` | Built-in | - | 数据仓库查询 |
| Spanner | `SpannerToolset` | Built-in | - | 分布式数据库 |
| **Built-in Tools - Third-party** |
| Github | Github工具集 | Built-in | - | 代码仓库管理 |
| HuggingFace | HF工具集 | Built-in | - | AI模型调用 |
| **Session Management** |
| Session Service | `InMemorySessionService` | 会话管理 | ✅ | 多轮对话上下文 |
| Tool Context | `ToolContext` | 上下文访问 | 📝 说明 | 访问会话状态 |

**图例**:
- ✅ 已导入并使用
- 📝 已说明用法
- - 未在本课程中使用

### ✅ 工具设计最佳实践

1. **单一职责原则**：每个工具专注做一件事
2. **清晰的命名**：工具名称要能直接表达功能
3. **完整的文档**：Docstring要详细说明参数和返回值
4. **结构化返回值**：使用字典格式，包含success/error信息
5. **业务逻辑验证**：在工具内部进行必要的检查
6. **工具组合使用**：根据需求合理组合Custom和Built-in工具
7. **注意兼容性**：某些工具不能混用（如google_search与自定义函数）

## 4.4 下一步学习建议

- 探索更复杂的多Agent协作模式
- 学习如何将Agent部署到生产环境
- 了解Agent的监控和调试技巧
- 研究安全性和权限控制
- 尝试集成Google Cloud Tools
- 探索MCP Tools的标准化工具开发

In [94]:
# 🎉 Day2 实验完成！
print("=" * 60)
print("🎉 恭喜！Day2 Agent Tools 实验完成！")
print("=" * 60)
print("\n✅ 你已经掌握了:")
print("   1. 创建自定义工具函数")
print("   2. 使用AgentTool封装子Agent")
print("   3. InMemorySessionService会话管理")
print("   4. 构建多工具Agent系统")
print("\n📝 下一步:")
print("   - 尝试创建更复杂的自定义工具")
print("   - 探索更多Agent协作模式")
print("   - 继续Day3的学习")
print("=" * 60)

🎉 恭喜！Day2 Agent Tools 实验完成！

✅ 你已经掌握了:
   1. 创建自定义工具函数
   2. 使用AgentTool封装子Agent
   3. InMemorySessionService会话管理
   4. 构建多工具Agent系统

📝 下一步:
   - 尝试创建更复杂的自定义工具
   - 探索更多Agent协作模式
   - 继续Day3的学习
